In [2]:
import cobra
import os
import pandas as pd
import riptide
import ipywidgets as widgets
import escher
import numpy as np
from pybiouml import pybiouml
from escher import Builder

joiner = lambda e1, e2: os.path.join(e1, e2)
maker = lambda l: [os.makedirs(el) for el in l]


# Prepare files and data

### Paths

In [4]:
path = '20Z'
initial_transcript = joiner(path, 'initial_transcriptomics')
wt_models = joiner(path, 'wt_models')
transcript = joiner(path, 'transcriptomics')
counts = joiner(path, 'counts')
out_path = joiner(path, 'out')
drf = joiner(out_path, 'DRF')
fluxes = joiner(out_path, 'fluxes')
out_models = joiner(out_path, 'models')

paths = [initial_transcript, transcript, out_path, counts, wt_models, drf, fluxes, out_models]

### Make file system

In [18]:
maker([path])
maker(paths)

[None, None, None, None, None, None, None, None]

### Download DEGs files

In [14]:
from pybiouml import pybiouml as pb
con = pb.PyBiouml()

In [23]:
degs_analysis_folder = con.ls('data/Collaboration (git)/20ZR_CS_GSM_model/Data/RNA-seq/Differential Gene Expression Analysis')
degs_analysis_folder

,name,hasChildren,class
0,"CH3OH (Ca,W,no_Cu) vs CH4 (Ca,W,Cu)",True,0
1,"CH3OH (La,W,no_Cu) vs CH4 (Ca,W,Cu)",True,0
2,"CH3OH Ca vs CH3OH La (W,no_Cu)",True,0
3,KEGG Pathway Enrichment Analysis,True,0
4,"Low CH4 (Ca,W,Cu) vs CH4 (Ca,W,Cu)",True,0
5,"Low CH4 (La,W,Cu) vs CH4 (Ca,W,Cu)",True,0


In [24]:
degs_folders = [f for f in degs_analysis_folder.name.values if not f.startswith('KEGG')]
degs_folders

['CH3OH (Ca,W,no_Cu) vs CH4 (Ca,W,Cu)',
 'CH3OH (La,W,no_Cu) vs CH4 (Ca,W,Cu)',
 'CH3OH Ca vs CH3OH La (W,no_Cu)',
 'Low CH4 (Ca,W,Cu) vs CH4 (Ca,W,Cu)',
 'Low CH4 (La,W,Cu) vs CH4 (Ca,W,Cu)']

In [26]:
for deg in degs_folders:
    con.export(f'data/Collaboration (git)/20ZR_CS_GSM_model/Data/RNA-seq/' \
        f'Differential Gene Expression Analysis/{deg}/{deg}.result',
                   exporter='Tab-separated text (*.txt)',
                   target_file=f'20Z/initial_transcriptomics/{deg}.txt')

### Download counts files

In [16]:
path_to_counts = 'data/Collaboration (git)/20ZR_CS_GSM_model/Data/RNA-seq/Counts'
counts_biouml_folder = con.ls(path_to_counts)
counts_biouml_folder

,name,hasChildren,class
0,normalized_counts.ch4,False,0
1,normalized_counts.ch4_lim,False,0
2,normalized_counts.la_ch4_bioreactor,False,0
3,normalized_counts.la_limit_bioreactor,False,0


In [20]:
counts_files = counts_biouml_folder['name'].tolist()
counts_files

['normalized_counts.ch4',
 'normalized_counts.ch4_lim',
 'normalized_counts.la_ch4_bioreactor',
 'normalized_counts.la_limit_bioreactor']

In [21]:
for cf in counts_files:
    con.export(f'{path_to_counts}/{cf}', 
               exporter='Tab-separated text (*.txt)', 
               target_file=f'20Z/counts/{cf}.txt')

In [23]:
con.logout()

### Prepare DEGs files to RIPTiDE format files

In [28]:
for e in os.listdir(initial_transcript):
    if not e.startswith('.'):
        t = pd.read_table(joiner(initial_transcript, e), index_col=0)
        t['log2FoldChange'].to_csv(joiner(transcript, e),sep='\t')

# Transcript integration in GSM model iIA409

In [13]:
model = cobra.io.read_sbml_model(joiner(path, 'W_La_Ca_iIA409_20Z.xml'))

CobraSBMLError: Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `cobra.io.sbml.validate_sbml_model` function or via the online validator at http://sbml.org/validator .
	`(model, errors) = validate_sbml_model(filename)`
If the model is valid and cannot be read please open an issue at https://github.com/opencobra/cobrapy/issues .

In [27]:
tr_types_files = [transcript, counts]

transcript_type_selector = widgets.Dropdown(
    options=tr_types_files,
    value=tr_types_files[0],
    description='Type of trancripts',
)
transcript_type_selector

Dropdown(description='Type of trancripts', options=('20Z/transcriptomics', '20Z/counts'), value='20Z/transcrip…

In [59]:
files = os.listdir(transcript_type_selector.value)

In [6]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001502,0,0.00%
ch4_e,EX_ch4_e,11.7,1,100.00%
cu2_e,EX_cu2_e,0.001502,0,0.00%
fe2_e,EX_fe2_e,0.000919,0,0.00%
mg2_e,EX_mg2_e,0.01051,0,0.00%
no3_e,EX_no3_e,1.019,0,0.00%
o2_e,EX_o2_e,13.77,0,0.00%
pi_e,EX_pi_e,0.1608,0,0.00%
so4_e,EX_so4_e,0.01772,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


In [60]:
transcript_selector = widgets.Dropdown(
    options=files,
    value=files[0],
    description='Transcripts',
)
transcript_selector

Dropdown(description='Transcripts', options=('control_2020.counts.tsv', 'La_W_Cu.counts.tsv', 'La_W_noCu_CH3OH…

In [41]:
choose_exchanges = widgets.SelectMultiple(
    options=[e.id for e in model.exchanges],
    description='Exchanges',
    disabled=False
)
choose_exchanges

SelectMultiple(description='Exchanges', options=('EX_ch4_e', 'EX_o2_e', 'EX_meoh_e', 'EX_c2h6_e', 'EX_h2o_e', …

In [42]:
tab_contents = choose_exchanges.value
children = [widgets.Text(description=name) for name in tab_contents]
tab_ex = widgets.Tab()
tab_ex.children = children
for i, n in enumerate(children):
    tab_ex.set_title(i, n.description)
tab_ex

In [43]:
bofs = [e.id for e in model.reactions.BOF.metabolites]
ex = ['_'.join([e.id.split('_')[1], 'c']) for e in model.exchanges]

for e in ex:
    if ex not in bofs:
        bofs.append(e)

In [44]:
choose_biomass_changes = widgets.SelectMultiple(
    options=bofs,
    description='Biomass metabolits',
    disabled=False
)
choose_biomass_changes

SelectMultiple(description='Biomass mets', options=('h2o_c', 'atp_c', 'nadh_c', 'nad_c', 'for_c', 'ac_c', 'coa…

In [45]:
tab_biomass = choose_biomass_changes.value
children_biomass = [widgets.Text(description=name) for name in tab_biomass]
tab_bio = widgets.Tab()
tab_bio.children = children_biomass
for i, n in enumerate(children_biomass):
    tab_bio.set_title(i, n.description)
tab_bio

In [13]:
change_bio = widgets.Checkbox(value=False,
                              description='Change biomass?',
                              disabled=False)
change_bio

Checkbox(value=False, description='Change biomass?')

In [61]:
name_selector = widgets.Text(
    value=transcript_selector.value,
    description='Name:',
    disabled=False
)
name_selector

Text(value='Ca_noW_noCu_CH3OH.counts.tsv', description='Name:')

In [62]:
def changing_model_for_conditions(model, file_name):
    changing_model = model.copy()
    for e in tab_ex.children:
        name = e.description
        lb = float(e.value)
        if name == 'EX_ch4_e':
            changing_model.reactions.get_by_id(name).upper_bound = lb
        if name == 'EX_la_e':
            changing_model.reactions.get_by_id('LAt').lower_bound = -1000.
            changing_model.reactions.MXALa_for.upper_bound =  1000.
        if name == 'EX_tungs_e':
            changing_model.reactions.get_by_id('TUNGSt').lower_bound = -1000.
        changing_model.reactions.get_by_id(name).lower_bound = lb
        
        
    if change_bio.value:
        bof = model.reactions.BOF
        bof_metabolites = bof.metabolites
        
        for bio in tab_bio.children:
            metabolite = bio.description
            st = float(bio.value)
            c_metabolite = model.metabolites.get_by_id(metabolite)
            if lb == 0:
                if c_metabolite in bof_metabolites.keys():
                    del bof_metabolites[c_metabolite]
            else:
                bof_metabolites[c_metabolite] = st

        changing_model.reactions.BOF.remove_from_model()
        
        new_bof = cobra.Reaction('BOF')
        new_bof.name = bof.name
        new_bof.lower_bound = 0.
        new_bof.upper_bound = 1000.
        new_bof.add_metabolites(bof_metabolites)
        changing_model.add_reactions([new_bof])   
        changing_model.objective = 'BOF'
        changing_model.problem.Constraint(model.reactions.get_by_id("BOF").flux_expression)
        
    cobra.io.write_sbml_model(changing_model, joiner(path, f'wt_models/{file_name}_wt.xml'))
    cobra.io.save_json_model(changing_model, joiner(path, f'wt_models/{file_name}_wt.json'))


In [66]:
changing_model_for_conditions(model, name_selector.value)

In [64]:
def run_riptide(file_name, frac_min, header=False, norm=True):
    changed_model = cobra.io.read_sbml_model(joiner(path, f'wt_models/{file_name}_wt.xml'))
    file_path = joiner(transcript_type_selector.value, transcript_selector.value)
    t1 = riptide.read_transcription_file(file_path, header=header, norm=norm)
    print(file_name, end='\n\n')
    
    o = riptide.maxfit(model=changed_model, transcriptome=t1, frac_min=frac_min, frac_max=1)

    cobra.io.write_sbml_model(o.model, joiner(out_path, f'models/{file_name}_{frac_min}.xml'))
    cobra.io.save_json_model(o.model, joiner(out_path, f'models/{file_name}_{frac_min}.json'))
    
    return o

In [67]:
cs_model = run_riptide(name_selector.value, frac_min=0.3, header=False, norm=False)

counts_Ca_W_noCu_CH3OH_2017

Running max fit RIPTiDe for objective fraction range: 0.3 to 0.9...
Analyzing context-specific subnetwork flux ranges...
Progress: 100%        

No level of optimal objective flux correlated with transcriptome
0.3 of optimum-associated model returned

Reactions pruned to 397 from 444 (10.59% change)
Metabolites pruned to 409 from 430 (4.88% change)

RIPTiDe completed in 12 seconds



AttributeError: 'str' object has no attribute 'to_csv'

In [10]:
wt_json_models = os.listdir('20Z/wt_models')
wt_model_selector = widgets.Dropdown(
    options=[f for f in wt_json_models if f.endswith('.json')],
    value=wt_json_models[0],
    description='Models',
)
wt_model_selector

Dropdown(description='Models', options=('oxygen_limitation_wt.json', 'Ca_noW_noCu_CH3OH_wt.json', 'La_noW_noCu…

In [3]:
builder_wt = Builder(map_json='20Z/new_20Zmap_La_W.json')
builder_wt.model = cobra.io.load_json_model(f'20Z/wt_models/{wt_model_selector.value}')
solution_wt = cobra.flux_analysis.pfba(builder_wt.model)
builder_wt.reaction_data = solution_wt.fluxes
builder_wt.reaction_scale = [
    { 'type': 'value', 'value': 10, 'color': '#ed3124', 'size': 25 },
    { 'type': 'value', 'value': 8, 'color': '#edb724', 'size': 20 },
    { 'type': 'value', 'value': 5, 'color': '#edea24', 'size': 20 },
    { 'type': 'value', 'value': 2.5, 'color': '#069108', 'size': 20 },
    { 'type': 'value', 'value': 1.0, 'color': '#022bf7', 'size': 20 },
    { 'type': 'value', 'value': 0.5, 'color': '#0299f7', 'size': 20 },
    { 'type': 'value', 'value': 0.05, 'color': '#9696ff', 'size': 16 },
    { 'type': 'value', 'value': 0, 'color': '#ededed', 'size': 16 }
    
]

In [4]:
builder_wt

Builder(reaction_data={'EX_ch4_e': -11.7, 'EX_o2_e': -13.771798687426847, 'EX_meoh_e': 0.0, 'MEohtep': 0.0, 'E…

In [51]:
path_to_models = joiner(out_path, 'models/')
json_models = [f for f in os.listdir(path_to_models) if f.endswith('.json')]

model_selector = widgets.Dropdown(
    options=json_models,
    value=json_models[0],
    description='Models',
)
model_selector

Dropdown(description='Models', options=('control_2020.counts.json', 'counts_methane_limitation.json', 'counts_…

In [52]:
builder_transcript = Builder(map_json='20Z/new_20Zmap_La_W.json')
builder_transcript.model = cobra.io.load_json_model(joiner(path_to_models, model_selector.value))
solution_transcript = cobra.flux_analysis.pfba(builder_transcript.model)
builder_transcript.reaction_data = solution_transcript.fluxes
builder_transcript.reaction_scale = [
    { 'type': 'value', 'value': 10, 'color': '#ed3124', 'size': 25 },
    { 'type': 'value', 'value': 8, 'color': '#edb724', 'size': 20 },
    { 'type': 'value', 'value': 5, 'color': '#edea24', 'size': 20 },
    { 'type': 'value', 'value': 2.5, 'color': '#069108', 'size': 20 },
    { 'type': 'value', 'value': 1.0, 'color': '#022bf7', 'size': 20 },
    { 'type': 'value', 'value': 0.5, 'color': '#0299f7', 'size': 20 },
    { 'type': 'value', 'value': 0.05, 'color': '#9696ff', 'size': 16 },
    { 'type': 'value', 'value': 0, 'color': '#ededed', 'size': 16 }
    
]

Builder(reaction_data={'EX_o2_e': -7.705504613049158, 'EX_meoh_e': -11.686541863753668, 'MEohtep': 11.68654186…

In [ ]:
builder_transcript

## DRF analysis

In [9]:
path_wt_models = ('20Z/wt_models')
path_to_models = joiner(out_path, 'models/')

wt_models = [f for f in os.listdir(path_wt_models) if f.endswith('.xml')]
cs_models = [f for f in os.listdir(path_to_models) if f.endswith('.xml')]

In [10]:
def model_1_selector_func(model_type):
    if model_type == 'cs':
        models = cs_models
    else:
        models = wt_models
    model_1_selector = widgets.Dropdown(
        options=models,
        value=models[0],
        description='Models',
    )
    return model_1_selector


model_1_selector = model_1_selector_func('cs')
model_1_selector

Dropdown(description='Models', options=('CS_model_ch4_Ca_W_Cu.xml', 'CS_model_La_W_Cu.xml'), value='CS_model_c…

In [11]:
model_2_selector = model_1_selector_func('cs')
model_2_selector

Dropdown(description='Models', options=('CS_model_ch4_Ca_W_Cu.xml', 'CS_model_La_W_Cu.xml'), value='CS_model_c…

In [54]:
def make_merged_fluxes(m1, m2, name1, name2):
    fluxes1 = cobra.flux_analysis.pfba(m1).fluxes
    fluxes2 = cobra.flux_analysis.pfba(m2).fluxes
    fluxes1.name = name1
    fluxes2.name = name2

    flux_df = pd.DataFrame([fluxes1, fluxes2]).T
    flux_df.dropna(subset=[name1, name2], inplace=True, how='all')
    flux_df.fillna(0.0, inplace=True)
    return flux_df


name1, name2 = 'CS_Ca_W_Cu', 'CS_La_W_Cu'

#path_wt_models - при использовании wt моделей iIA409
#path_to_models - при использование CS-моделей

m1 = cobra.io.read_sbml_model(joiner(path_to_models, model_1_selector.value))
m2 = cobra.io.read_sbml_model(joiner(path_wt_models, model_2_selector.value))

flux_df = make_merged_fluxes(m1, m2, name1, name2)

In [55]:
col_name = 'DRF'

for i, r in flux_df.iterrows():
    if (r[name2]) != 0:
        if r[name1] != 0:
            flux_df.loc[i, col_name] = round(r[name1] / r[name2], 5)
        else:
            flux_df.loc[i, col_name] = 1000.0
    else:
        if r[name1] != 0:
            flux_df.loc[i, col_name] = 2000.0
        else:
            flux_df.loc[i, col_name] = 0.0
            
#fluxes
filt_table = flux_df[(flux_df[col_name] >= 1.5) | (flux_df[col_name] < 0.67)]
filt_table.to_csv(f'{out_path}/DRF/{name1}_vs_{name2}.txt', sep='\t')
filt_table[col_name].to_csv(f'{out_path}/DRF/fluxes_{name1}_vs_{name2}.csv', sep=',')

In [ ]:
scale_dict = [
    { 'type': 'value', 'value': 10, 'color': '#ed3124', 'size': 25 },
    { 'type': 'value', 'value': 8, 'color': '#edb724', 'size': 20 },
    { 'type': 'value', 'value': 5, 'color': '#edea24', 'size': 20 },
    { 'type': 'value', 'value': 2.5, 'color': '#069108', 'size': 20 },
    { 'type': 'value', 'value': 1.0, 'color': '#022bf7', 'size': 20 },
    { 'type': 'value', 'value': 0.5, 'color': '#0299f7', 'size': 20 },
    { 'type': 'value', 'value': 0.05, 'color': '#9696ff', 'size': 16 },
    { 'type': 'value', 'value': 0, 'color': '#ededed', 'size': 16 },
    { 'type': 'value', 'value': 1000.0, 'color': '#ff00c3', 'size': 25 },
    { 'type': 'value', 'value': 2000.0, 'color': '#00fffb', 'size': 25 }
]

In [56]:
builder_def = Builder(map_json='20Z/new_20Zmap_La_W.json')
builder_def.reaction_data = filt_table[col_name]
builder_def.reaction_scale = scale_dict
builder_def.save_html(f'{out_path}/DRF/{name1}_vs_{name2}.html')

In [57]:
f'{name1}_vs_{name2}'

'Ca_W_noCu_CH3OH_2017_vs_methane_limitation_Ca'

In [58]:
builder_def

Builder(reaction_data={'EX_o2_e': 0.62104, 'EX_meoh_e': 2000.0, 'MEohtep': 2000.0, 'O2tec': 2.38753, 'H2Otcp':…